In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader

In [ ]:
data_or=pd.read_csv('train.csv')
data_or

In [ ]:
def tran(x):
    x=x.split(' ')
    templis=[]
    for i in range(len(x)):
        templis.append(int(x[i]))
    templis=np.array(templis)
    np_fin=templis.reshape(48,48)
    return np_fin
x=tran(data_or.iloc[0][1])
x.shape
        

In [ ]:
templis=[]
for i in range(2304):
    templis.append(int(x1[i]))
len(templis)
nplis=np.array(templis)
nplis=nplis.reshape(48,48)
nplis.shape
plt.imshow(nplis, cmap='gray')  # 使用灰度颜色映射
plt.colorbar()  # 显示颜色条
plt.show()

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
# 如果需要加载图像，可以取消下一行的注释
# from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_or, tran,transform=None):

        self.data_or = data_or
        self.transform = transform
        self.tran=tran
    def __len__(self):

        return len(self.data_or)

    def __getitem__(self, idx):

        x=data_or.iloc[idx][1]
        x=tran(x)
        y=data_or.iloc[idx][0]
        return x,y


In [ ]:
dataset = CustomDataset(data_or=data_or,tran=tran)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)
for x,y in dataloader:
    print(x,y)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchEmbeddings(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, embed_size):
        super(PatchEmbeddings, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embed_size = embed_size

        self.n_patches = (img_size // patch_size) ** 2
        self.projection = nn.Conv2d(in_channels, embed_size, kernel_size=patch_size, stride=patch_size)

        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_size))
        self.positions = nn.Parameter(torch.randn(self.n_patches + 1, embed_size))

    def forward(self, x):
        B = x.shape[0]
        x = self.projection(x)  # [B, E, H', W']
        x = x.flatten(2)  # Flatten patches [B, E, N_patches]
        x = x.transpose(1, 2)  # Transpose to get [B, N_patches, E]
        
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)  # Concatenate cls_token
        x += self.positions  # Add position embeddings
        return x


class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(TransformerEncoderLayer, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        identity = x
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(attn_output + identity)
        x = self.dropout(x)
        forward = self.feed_forward(x)
        x = self.norm2(forward + x)
        x = self.dropout(x)
        return x

class ViT(nn.Module):
    def __init__(self, img_size=48, patch_size=12, in_channels=1, num_classes=1000,
                 embed_size=768, depth=12, heads=8, forward_expansion=4, dropout=0.1):
        super(ViT, self).__init__()
        self.patch_embeddings = PatchEmbeddings(img_size, patch_size, in_channels, embed_size)
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(embed_size, heads, forward_expansion, dropout) 
            for _ in range(depth)
        ])
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_size),
            nn.Linear(embed_size, num_classes)
        )

    def forward(self, x):
        x = self.patch_embeddings(x)
        for layer in self.encoder_layers:
            x = layer(x)
        cls_token = x[:, 0]
        out = self.mlp_head(cls_token)
        return out
model=ViT()
x = torch.randn(1, 1, 48, 48)
print(model(x).shape)

torch.Size([1, 1000])


In [1]:
import torch

x = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

# 没有指定dim，返回所有元素的最大值
max_value = torch.max(x)
print("Max value:", max_value)

# 指定dim=0，沿着第一个维度（垂直方向）计算最大值
max_values, indices = torch.max(x, dim=0)
print("Max values along dim=0:", max_values)
print("Indices of max values along dim=0:", indices)

# 指定dim=1，沿着第二个维度（水平方向）计算最大值
max_values, indices = torch.max(x, dim=1)
print("Max values along dim=1:", max_values)
print("Indices of max values along dim=1:", indices)


Max value: tensor(6)
Max values along dim=0: tensor([4, 5, 6])
Indices of max values along dim=0: tensor([1, 1, 1])
Max values along dim=1: tensor([3, 6])
Indices of max values along dim=1: tensor([2, 2])
